In [382]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import sqlite3 as sql

In [383]:
dtCarga = datetime.today().strftime('%d/%m/%Y %H:%M')

# COLETA DE DADOS DO SITE DO IBGE

In [384]:
#COLETANDO DADOS DO IBGE
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#RJ'

DadosIBGE = pd.DataFrame(pd.read_html(url, match="Municípios do Rio de Janeiro")[0])

#RENOMEAR COLUNAS DO DF
DadosIBGE = DadosIBGE.rename(columns={'Municípios do Rio de Janeiro':'nmMunic','Códigos':'codMunic'})

#ALTERANDO NOME DO INDEX
DadosIBGE.index.name = 'idMunic'

#ALTERANDO INDEX PRA COMEÇAR EM 1
DadosIBGE.index = DadosIBGE.index + 1

#ADICIONAR UM COLUNA DE DATA E HORA DE CARGA
DadosIBGE['dtCarga'] = dtCarga

#DadosIBGE

# CRIANDO O BANCO DE DADOS ODS

In [385]:
#Manipulando o sistema de arquivos
endereco = Path('C:\\Users\\claud\\OneDrive\\Claudio Bonel-DADOTECA\\Projetos Sociais\\Projeto Dado Humanizado\\Pascoa com Dados\\')

BDODS = endereco / "pascoaODS.db"
BDDW = endereco / "pascoaDW.db"

if endereco.exists():
    if (BDODS.exists() and BDDW.exists()):
        print('Bancos de dados já existem!')
    else:
        BDODS.touch()
        BDDW.touch()
        print('Bancos de dados criados!')
else:
    print('Endereço não existe! Favor, verificar!')
        

Bancos de dados já existem!


# MANIPULANDO OS BANCOS DE DADOS CRIADOS

In [386]:
#Conectar no BDODS
conexaoODS = sql.connect(BDODS)

#Criar a tabela tbLogMunic e carregar os dados do DF DadosIBGE
DadosIBGE.to_sql('tbLogMunic',conexaoODS,if_exists="append")

#Confirmar a transação
conexaoODS.commit()

#Fechar a conexão
conexaoODS.close()

print('Carga do BDODS concluída!')

Carga do BDODS concluída!


In [387]:
#Conectar no BDDW
conexaoDW = sql.connect(BDDW)

#Selecionar somente as colunas para criação da dMunicipio
DadosIBGE = DadosIBGE[['nmMunic','codMunic']]

#Criar a tabela dMunicipio e carregar os dados do DF DadosIBGE
DadosIBGE.to_sql('dMunicipio',conexaoDW,if_exists="replace")

#Confirmar a transação
conexaoDW.commit()

#Fechar a conexão
conexaoDW.close()

print('Carga do DW concluída!')

Carga do DW concluída!


# CARGA DOS DADOS DE CADASTRO DAS DELEGACIAS DE POLÍCIA

In [388]:
#Definir o endereço físico do arquivo CSV
LocalArquivo = "C:\\Users\\claud\\OneDrive\\Claudio Bonel-DADOTECA\\CEPERJ\\Aulas\\Dados\\DADOS_POS\\"

#Criar Data Frame com os dados das DPs
tbLogDP = pd.DataFrame(pd.read_csv(LocalArquivo + "DP.csv"))

#Criando coluna com a data da carga
tbLogDP['dtCarga'] = dtCarga

#selecionando a coluna COD_DP como novo index
#tbLogDP = tbLogDP.set_index("COD_DP")

# CARGA DOS DADOS DE CADASTRO DOS RESPONSÁVEIS PELAS DELEGACIAS DE POLÍCIA

In [389]:
#Criar Data Frame com os dados dos responsáveis pelas DPs
tbLogRespDP = pd.DataFrame(pd.read_csv(LocalArquivo + "ResponsavelDP.csv"))

#Criando coluna com a data da carga
tbLogRespDP['dtCarga'] = dtCarga

#selecionando a coluna COD_DP como novo index
#tbLogRespDP = tbLogRespDP.set_index("COD_DP")

# CONECTANDO NO ODS E INFORMANDO AO PYTHON QUE USAREI LINGUAGEM SQL

In [390]:
#Conectar no BDODS
conexaoODS = sql.connect(BDODS)

#Definição de manipulação de dados em SQL
SQL_ODS = conexaoODS.cursor()

# INSERINDO DADOS DAS DELEGACIAS DE POLÍCIA, NO SQLITE (ODS) 

In [391]:
#Inserindo registros na tabela tbLogDP
print("Iniciando a inserção de dados na tabela tbLogDP.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogDP (codDP,nmDP,endereco,dtCarga) VALUES (?,?,?,?)''',tbLogDP.values.tolist())

#confirmar a transação
conexaoODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogDP), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela tbLogDP.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na tbLogDP!


# INSERINDO DADOS DOS RESPONSÁVEIS PELAS DELEGACIAS DE POLÍCIA, NO SQLITE (ODS) 

In [392]:
#Inserindo registros na tabela tbLogDP
print("Iniciando a inserção de dados na tabela tbLogRespDP.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogRespDP (codDP,nmResponsavel,dtCarga) VALUES (?,?,?)''',tbLogRespDP.values.tolist())

#confirmar a transação
conexaoODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogRespDP), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela tbLogRespDP.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na tbLogDP!


# VALIDANDO CARGA DOS DADOS DE DP E DE RESPONSÁVEL DP

In [393]:
#validar a carga através de um select na tblogDP
#pd.read_sql("SELECT * FROM tbLogDP",conexaoODS)

In [394]:
#validar a carga através de um select na tblogRespDP
#pd.read_sql("SELECT * FROM tbLogRespDP",conexaoODS)

# CRIANDO O DATAFRAME COM O SELECT DE CRIAÇÃO DA DIMENSÃO DP (dDP), NO DW

In [395]:
#Definir a query de criação da dimensão dDP

qry_dDP = '''
    SELECT 
        codDP,
        nmDP,
        endereco,
        nmResponsavel
    FROM
    (
        SELECT
            a.codDP,
            a.nmDP,
            a.endereco,
            b.nmResponsavel,
            max(a.dtCarga)
        FROM tbLogDP a
        JOIN tbLogRespDP b
        on a.codDP = b.codDP
        WHERE a.dtCarga = (SELECT max(x.dtCarga) FROM tbLogDP x)
        GROUP BY
            a.codDP,
            a.nmDP,
            a.endereco,
            b.nmResponsavel
    ) a
'''

dDP = pd.read_sql(qry_dDP,conexaoODS)

# COLETANDO DADOS DOS BATALHÕES DE POLÍCIA MILITAR (BPM)

In [396]:
#Criar Data Frame com os dados dos BPMs
tbLogBPM = pd.DataFrame(pd.read_excel(LocalArquivo + "BPM.xlsx"))

#Criando coluna com a data da carga
tbLogBPM['dtCarga'] = dtCarga


# CONSTRUINDO A TABELA DE LOG DE REGISTROS DO BPM (tbLogBPM), no ODS

In [397]:
#Definir a query de criação da tabela
qry_tbLogBPM = '''
    
    CREATE TABLE IF NOT EXISTS tbLogBPM (
    
    codBPM INTEGER,
    nmBPM VARCHAR (7),
    enderecoBPM VARCHAR (200),
    dtCarga DATETIME

    )
'''

#executar a criação da tbLogBPM
SQL_ODS.execute(qry_tbLogBPM)

#definição da qry de criação de index
qry_idx_tbLogBPM_codBPM = "CREATE INDEX IF NOT EXISTS idx_tbLogBPM_codBPM ON tbLogBPM (codBPM)"

#executar a criação do index
SQL_ODS.execute(qry_idx_tbLogBPM_codBPM)

# INSERINDO DADOS DE BPM NA TABELA tbLogBPM

In [398]:
#Inserindo registros na tabela tbLogBPM
print("Iniciando a inserção de dados na tabela tbLogBPM.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogBPM (codBPM,nmBPM,enderecoBPM,dtCarga) VALUES (?,?,?,?)''',tbLogBPM.values.tolist())

#confirmar a transação
conexaoODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogBPM), "registros inseridos na tbLogBPM!")

Iniciando a inserção de dados na tabela tbLogBPM.
-------------------------------------------------
Carga finalizada! 41 registros inseridos na tbLogBPM!


# COLETANDO DADOS DA ÁREA DOS BATALHÕES DE POLÍCIA MILITAR (BPM)

In [399]:
#Criar Data Frame com os dados dos BPMs
tbLogAreaBPM = pd.DataFrame(pd.read_csv(LocalArquivo + "areaBPMv1.csv"))

#Criando coluna com a data da carga
tbLogAreaBPM['dtCarga'] = dtCarga


# CONSTRUINDO A TABELA DE LOG DE REGISTROS DA ÁREA DOS BPMs (tbLogAreaBPM), no ODS

In [400]:
#Definir a query de criação da tabela
qry_tbLogAreaBPM = '''
    
    CREATE TABLE IF NOT EXISTS tbLogAreaBPM (
    
    codBPM INTEGER,
    areaBPM REAL (5,2),
    dtCarga DATETIME

    )
'''

#executar a criação da tbLogAreaBPM
SQL_ODS.execute(qry_tbLogAreaBPM)

#definição da qry de criação de index
qry_idx_tbLogAreaBPM_codBPM = "CREATE INDEX IF NOT EXISTS idx_tbLogAreaBPM_codBPM ON tbLogAreaBPM (codBPM)"

#executar a criação do index
SQL_ODS.execute(qry_idx_tbLogAreaBPM_codBPM)

# INSERINDO DADOS DA ÁREA DOS BPMs NA TABELA tbLogAreaBPM

In [401]:
#Inserindo registros na tabela tbLogAreaBPM
print("Iniciando a inserção de dados na tabela tbLogAreaBPM.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogAreaBPM (codBPM,areaBPM,dtCarga) VALUES (?,?,?)''',tbLogAreaBPM.values.tolist())

#confirmar a transação
conexaoODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogAreaBPM), "registros inseridos na tbLogAreaBPM!")

Iniciando a inserção de dados na tabela tbLogAreaBPM.
-------------------------------------------------
Carga finalizada! 39 registros inseridos na tbLogAreaBPM!


# CRIANDO O DATAFRAME COM O SELECT DE CRIAÇÃO DA DIMENSÃO DP (dBPM), NO DW

In [402]:
#Definir a query de criação da dimensão dBPM

qry_dBPM = '''
    SELECT
        codBPM,
        nmBPM,
        enderecoBPM,
        areaBPM
    FROM 
    (
        SELECT 
            a.codBPM,
            a.nmBPM,
            a.enderecoBPM,
            b.areaBPM,
            max(a.dtCarga)
        FROM tbLogBPM a
        JOIN tbLogAreaBPM b
        ON a.codBPM = b.codBPM
        WHERE a.dtCarga = (
                            SELECT MAX(x.dtCarga)
                            FROM tbLogBPM x
                       )
        GROUP BY
            a.codBPM,
            a.nmBPM,
            a.enderecoBPM,
            b.areaBPM
    ) a
'''

dBPM = pd.read_sql(qry_dBPM,conexaoODS)

# FECHANDO A CONEXÃO COM O ODS

In [403]:
#Confirmar a transação
conexaoODS.commit()

#Fechar a conexão
conexaoODS.close()

# CONECTANDO NO DW E INFORMANDO AO PYTHON QUE USAREI LINGUAGEM SQL

In [404]:
#Conectar no BDODS
conexaoDW = sql.connect(BDDW)

#Definição de manipulação de dados em SQL
SQL_DW = conexaoDW.cursor()

# DELETANDO DADOS ANTIGOS E REINICIANDO AO AUTOINCREMENTAL DA CHAVE, DA TABELA dDP

In [405]:
#Deletar os dados atuais
SQL_DW.execute("DELETE FROM dDP")

#Reiniciando o autoincramental da chave
SQL_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dDP'")

#Confirmar a transação
conexaoDW.commit()

# INSERINDO DADOS NA DIMENSÃO DP (dDP), NO DW

In [406]:
#Inserindo registros na tabela dDP
print("Iniciando a inserção de dados na tabela dDP.")
print("-------------------------------------------------")

SQL_DW.executemany('''INSERT INTO dDP (codDP,nmDP,endereco,nmResponsavel) VALUES (?,?,?,?)''',dDP.values.tolist())

#confirmar a transação
conexaoDW.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dDP), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela dDP.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na tbLogDP!


# VALIDANDO CARGA DOS DADOS DA DIMENSÃO DP (dDP)

In [407]:
#validar a carga através de um select na tblogDP
#pd.read_sql("SELECT * FROM dDP",conexaoDW)

# CONSTRUINDO A TABELA DE DIMENSÃO BPM (dBPM), no DW

In [408]:
#Definir a query de criação da tabela
qry_dBPM = '''
    
    CREATE TABLE IF NOT EXISTS dBPM (
    
    idBPM INTEGER PRIMARY KEY AUTOINCREMENT,
    codBPM INTEGER,
    nmBPM VARCHAR (7),
    enderecoBPM VARCHAR (200),
    areaBPM REAL (5,2)

    )
'''

#executar a criação da tbLogBPM
SQL_DW.execute(qry_dBPM)

#executar a criação do index idBPM
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dBPM_idBPM ON dBPM (idBPM)")

#executar a criação do index codBPM
SQL_DW.execute("CREATE INDEX IF NOT EXISTS idx_dBPM_codBPM ON dBPM (codBPM)")

# DELETANDO DADOS ANTIGOS E REINICIANDO AO AUTOINCREMENTAL DA CHAVE, DA TABELA dBPM

In [409]:
#Deletar os dados atuais
SQL_DW.execute("DELETE FROM dBPM")

#Reiniciando o autoincramental da chave
SQL_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dBPM'")

#Confirmar a transação
conexaoDW.commit()

# INSERINDO DADOS NA DIMENSÃO BPM (dBPM), NO DW

In [410]:
#Inserindo registros na tabela dBPM
print("Iniciando a inserção de dados na tabela dBPM.")
print("-------------------------------------------------")

SQL_DW.executemany('''INSERT INTO dBPM (codBPM,nmBPM,enderecoBPM,areaBPM) VALUES (?,?,?,?)''',dBPM.values.tolist())

#confirmar a transação
conexaoDW.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dBPM), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela dBPM.
-------------------------------------------------
Carga finalizada! 39 registros inseridos na tbLogDP!


# FECHANDO A CONEXÃO COM O DW

In [411]:
#Confirmar a transação
conexaoDW.commit()

#Fechar a conexão
conexaoDW.close()